In [1]:
import torch
import torch.nn as nn

Написать на PyTorch forward и backward полносвязного слоя без использования autograd

In [2]:
class MyCustomLinear(nn.Module):
    def __init__(self, len_input, len_output):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(len_input, len_output))
        self.bias = nn.Parameter(torch.rand(1, len_input))
        self.x = None
        self.grad = None

    def forward(self, x):
        x = x @ self.weight + self.bias
        return x

    def backward(self, grad):
        grad = self.weight.t() @ self.grad
        return grad

Написать 1-2 адаптивных оптимизатора
1) Adagrad
accumulated += gradient ** 2
adapt_lr = lr / sqrt (accumulated)
w = w - adapt_lr * gradient

In [3]:
class MyAdagrad:
    def __init__(self, w, lr):
        self.lr = lr
        self.w = w

    def step(self, grad):
        accumulated = grad ** 2
        adapt_lr = self.lr / (accumulated**0.5)
        self.w = self.w - adapt_lr * grad
        return self.w

2) RMSprop
accumulated += rho (0.9 - 0.99) * accumulated + (1- rho) * gradient ** 2
adapt_lr = lr / sqrt (accumulated)
w = w - adapt_lr * gradient

Аккумуляция градиента за последние шаги, а не всегда как в Adagrad

In [4]:
class MyRMSProp:
    def __init__(self, w, lr, rho=0.9):
        self.accumulated = 0
        self.rho = rho
        self.lr = lr
        self.w = w
        if rho < 0.9 or rho > 0.99: return print('rho  will be >=  0.9 and <= 0.99')

    def step(self, grad):
        self.accumulated = self.rho * self.accumulated + (1 - self.rho) * grad ** 2
        adapt_lr = self.lr / (self.accumulated**0.5)
        self.w = self.w - adapt_lr * grad
        return self.w

Решить задачу нахождения корней квадратного уравнения методом градиентного спуска

In [5]:
class ZerosOfQuadricFunc:
    def __init__(self, a, b, c, x_init, optim, lr):
        self.a = a
        self.b = b
        self.c = c
        self.optim = optim
        self.lr = lr
        self.x_init = x_init
        if self.a == 0:
            print('Данное уравнение не является квадратным')

    def func_str(self):
        new_a = self.a
        new_b = self.b
        new_c = self.c
        if abs(self.a) == 1:
            if self.a > 0:
                new_a = 'x^2 '
            else:
                new_a = '- x^2 '
        else:
            new_a = str(new_a) + ' '
        if self.b > 0:
            if self.b == 1:
                new_b = '+ x '
            else:
                new_b = '+ ' + str(new_b)+'x '
        elif self.b < 0:
            if self.b == -1:
                new_b = '- x '
            else:
                new_b = '- ' + str(abs(new_b))+'x '
        else:
            new_b = ''
        if self.c > 0:
            if self.c == 1:
                new_c = '+ 1'
            else:
                new_c = '+ ' + str(new_c)
        elif self.c < 0:
            if self.c == -1:
                new_c = '- 1'
            else:
                new_c = '- ' + str(abs(new_c))
        else:
            new_c = ''
        return 'f(x) = ' + new_a + new_b + new_c

    def func_val(self, x):
        if (type(x) == type(None)) and (self.a == 0) and (self.peak() == None):
            return None
        else:
            y = self.a * x ** 2 + self.b * x + self.c
            return y

    def grad_func_val(self, x):
        if self.a == 0: return [None]
        grad_y = 2 * self.a * x + self.b
        return grad_y

    def peak(self):
        if self.a == 0:
            return None
        else:
            x = self.x_init
            optim = self.optim(x, self.lr)
            if self.grad_func_val(x) == 0:
                return x
            else:
                for i in range(100000):
                    x = optim.step([-self.grad_func_val(x) if self.a < 0 else self.grad_func_val(x)][0])
                return x

    def zeros(self):
        if (self.a == 0) or (self.peak() == None):
            return [None]
        else:
            y0 = self.func_val(self.peak())
            if self.a < 0:
                y0 = - y0
            if  y0 > 0.1e-5 or self.a == 0:
                print('Корней нет')
                return [None]
            elif abs(y0) < 0.1e-5:
                x0 = self.peak()
                return [x0]
            else:
                def grad_func2_val(x):
                    grad_y2 = 2 * (self.a * x ** 2 + self.b * x + self.c) * (2 * self.a * x + self.b)
                    return grad_y2
                x1 = self.peak() - 10*self.lr
                x2 = self.peak() + 10*self.lr
                optim = self.optim(x1, self.lr)
                for i in range(1000000):
                    x1 = optim.step(grad_func2_val(x1))
                optim = self.optim(x2, self.lr)
                for i in range(1000000):
                    x2 = optim.step(grad_func2_val(x2))
                return [x1, x2]

In [6]:
point_start = -9
zeros_search = ZerosOfQuadricFunc(a=1, b=3, c=15, x_init=point_start, optim=MyRMSProp, lr=0.001)
print('Найдем нули функции: ' + zeros_search.func_str())
print('Координаты точки, из которой идем искать корни  (' + str(point_start) + '; ' + str(zeros_search.func_val(point_start)) + ')')
print('Вершина в точке с координатами (' + str(zeros_search.peak()) + '; ' + str(zeros_search.func_val(zeros_search.peak())) + ')')
zeros = zeros_search.zeros()
if zeros[0] != None:
    if len(zeros) == 1:
        print('Квадратичная функция является полным квадратом суммы или разности')
        print('Корень уравнения: ' + str(zeros[0]))
    else:
        print('Корни уравнения: x1 = ' + str(zeros[0]) + '; x2 = ' + str(zeros[1]))

Найдем нули функции: f(x) = x^2 + 3x + 15
Координаты точки, из которой идем искать корни  (-9; 69)
Вершина в точке с координатами (-1.5; 12.75)
Корней нет


In [7]:
point_start = -1
zeros_search = ZerosOfQuadricFunc(a=-1, b=-2, c=+15, x_init=point_start, optim=MyRMSProp, lr=0.001)
print('Найдем нули функции: ' + zeros_search.func_str())
print('Координаты точки, из которой идем искать корни  (' + str(point_start) + '; ' + str(zeros_search.func_val(point_start)) + ')')
print('Вершина в точке с координатами (' + str(zeros_search.peak()) + '; ' + str(zeros_search.func_val(zeros_search.peak())) + ')')
zeros = zeros_search.zeros()
if zeros[0] != None:
    if len(zeros) == 1:
        print('Квадратичная функция является полным квадратом суммы или разности')
        print('Корень уравнения: ' + str(zeros[0]))
    else:
        print('Корни уравнения: x1 = ' + str(zeros[0]) + '; x2 = ' + str(zeros[1]))

Найдем нули функции: f(x) = - x^2 - 2x + 15
Координаты точки, из которой идем искать корни  (-1; 16)
Вершина в точке с координатами (-1; 16)
Корни уравнения: x1 = -5.0; x2 = 3.0


In [8]:
point_start = -9
zeros_search = ZerosOfQuadricFunc(a=0, b=3, c=15, x_init=point_start, optim=MyRMSProp, lr=0.001)
print('Найдем нули функции: ' + zeros_search.func_str())
print('Координаты точки, из которой идем искать корни  (' + str(point_start) + '; ' + str(zeros_search.func_val(point_start)) + ')')
print('Вершина в точке с координатами (' + str(zeros_search.peak()) + '; ' + str(zeros_search.func_val(zeros_search.peak())) + ')')
zeros = zeros_search.zeros()
if zeros[0] != None:
    if len(zeros) == 1:
        print('Квадратичная функция является полным квадратом суммы или разности')
        print('Корень уравнения: ' + str(zeros[0]))
    else:
        print('Корни уравнения: x1 = ' + str(zeros[0]) + '; x2 = ' + str(zeros[1]))

Данное уравнение не является квадратным
Найдем нули функции: f(x) = 0 + 3x + 15
Координаты точки, из которой идем искать корни  (-9; -12)
Вершина в точке с координатами (None; None)


In [9]:
point_start = -9
zeros_search = ZerosOfQuadricFunc(a=-1, b=0, c=0, x_init=point_start, optim=MyAdagrad, lr=0.001)
print('Найдем нули функции: ' + zeros_search.func_str())
print('Координаты точки, из которой идем искать корни  (' + str(point_start) + '; ' + str(zeros_search.func_val(point_start)) + ')')
print('Вершина в точке с координатами (' + str(zeros_search.peak()) + '; ' + str(zeros_search.func_val(zeros_search.peak())) + ')')
zeros = zeros_search.zeros()
if zeros[0] != None:
    if len(zeros) == 1:
        print('Квадратичная функция является полным квадратом суммы или разности')
        print('Корень уравнения: ' + str(zeros[0]))
    else:
        print('Корни уравнения: x1 = ' + str(zeros[0]) + '; x2 = ' + str(zeros[1]))

Найдем нули функции: f(x) = - x^2 
Координаты точки, из которой идем искать корни  (-9; -81)
Вершина в точке с координатами (4.517428098260723e-13; -2.0407156622955491e-25)
Квадратичная функция является полным квадратом суммы или разности
Корень уравнения: 4.517428098260723e-13
